In [5]:
import pymysql
import pandas as np
import random as rdm
import datetime
import time
start = time.time()

In [1]:
def createTable(name):
    row = [ 'tagname varchar(10)','site varchar(10)','rs_v varchar(10)','st_v varchar(10)','tr_v varchar(10)','r_a varchar(10)','s_a varchar(10)',
           't_a varchar(10)','kwh varchar(10)','pf varchar(10)','kw varchar(10)','is_on varchar(10)','datetime datetime']
    sql = "create table %s ("%(name)
    sql = sql+','.join(row)+")"
    return sql

In [2]:
createTable("point_info")

'create table point_info (tagname varchar(10),site varchar(10),rs_v varchar(10),st_v varchar(10),tr_v varchar(10),r_a varchar(10),s_a varchar(10),t_a varchar(10),kwh varchar(10),pf varchar(10),kw varchar(10),is_on varchar(10),datetime datetime)'

In [ ]:
import pymysql

# 打开数据库连接
db = pymysql.connect("localhost","root","j6t/6bj6","clientdb" )
 
# 使用 cursor() 方法创建一个游标对象 cursor
cursor = db.cursor()
 
# 使用 execute() 方法执行 SQL，如果表存在则删除
cursor.execute(createTable('points_info'))
 
# 关闭数据库连接
db.close()

In [6]:
def generate_insert(table,num):
    row = [ 'tagname','site','rs_v','st_v','tr_v','r_a','s_a','t_a','kwh','pf','kw','is_on','datetime']
    
    if(rdm.randint(0,100)>50):
        ison = 1
    else:
        ison = 0
    
    datas = {
        'tagname' : "A%s"%(str(num)),
        'site' : "N"+str(rdm.randint(0,10)),
        'rs_v' : 380,
        'st_v' : 380,
        'tr_v' : 380,
        'r_a' : rdm.randint(0,1000),
        's_a' : rdm.randint(0,1000),
        't_a' : rdm.randint(0,1000),
        'kwh' : rdm.randint(0,1000),
        'pf' : rdm.randint(0,1000),
        'kw' : rdm.randint(0,1000),
        'is_on' : ison,
        'datetime' : datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }
    template = "insert into %s ("%(table)
    template = template+','.join(row)+") values ("
    for col in row:
        template = template+"'"+str(datas[col])+"'"+','
    template = template[:-1]+')'
    return template

In [7]:
generate_insert('points_info',9)

"insert into points_info (tagname,site,rs_v,st_v,tr_v,r_a,s_a,t_a,kwh,pf,kw,is_on,datetime) values ('A9','N2','380','380','380','832','120','415','317','874','174','0','2019-07-19 13:21:14')"

In [8]:
def simulate_sql(db,cursor,table,points):
    for num in range(points):
        try:
            data = generate_insert(table,num+1)
            cursor.execute(data)
            db.commit()
        except pymysql.InternalError as e:
            db.rollback()
            print("error occurs : ",e)
            return 0
    return 1

In [5]:
db = pymysql.connect("localhost","root","j6t/6bj6","clientdb" )
# 使用 cursor() 方法创建一个游标对象 cursor
cursor = db.cursor()
simulate_sql(db,cursor,'point_info',9)
db.close()

In [9]:
db = pymysql.connect("localhost","root","j6t/6bj6","clientdb" )
# 使用 cursor() 方法创建一个游标对象 cursor
cursor = db.cursor()
# 使用 fetchone() 方法获取单条数据.
period = 10
count = 0
day = 7
point_num = 9
start = time.time()
value = day*86400/period*point_num
result = 1
limit = 100
print("point_num : %d , sleep time : %d , start time : %f , limit : %d" %(point_num,period,start,limit))
#print("%d days , %d points , there will be %d datas will be insert"%(day,point_num,day*86400/period*point_num))
while count<(day*86400/period):
    #print("Number %d data incoming."%(count+1))
    time.sleep(period)
    result = simulate_sql(db,cursor,'point_info',point_num)
    if not result:
        print("error occurs")
        break
    if count > limit:
        break
    count+=1
# 关闭数据库连接
end = time.time()
print('Insert finish , cost time : %d'%(end-start))
db.close()

point_num : 9 , sleep time : 10 , start time : 1563513907.000263 , limit : 100
Insert finish , cost time : 1024
